In [2]:
import unittest

In [25]:
# Arithmetic functions

def add(x, y):
    return x + y

def multiply(x, y):
    return x * y

def divide(x, y):
    if y == 0:
        raise ValueError('Can not divide by zero!')
    return x/y

[TestCase Assert Methods](https://docs.python.org/3/library/unittest.html#unittest.TestCase)


In [37]:
# Create a class and inherit from `unittest.TestCase`
class TestCalc(unittest.TestCase):
    
    def test_add(self):  # Testing methods must start with `test_`
        self.assertEqual(add(10, 5), 15)
        self.assertEqual(add(-1, 1), 0)
        self.assertEqual(add(-1, -1), -2)
        
    def test_multiply(self):
        self.assertEqual(multiply(2, 10), 20)
        self.assertEqual(multiply(10, 0), 0)
        self.assertEqual(multiply(4, 0.25), 1.0)
        
    def test_divide(self):
        self.assertEqual(divide(10, 2), 5)
        self.assertEqual(divide(-1, 1), -1)
        self.assertEqual(divide(-1, -1), 1)
        
        # Checking if dividing by 0 raises ValueError
        self.assertRaises(ValueError, divide, 10, 0)
        
        # With Context Manager (preferred)
        with self.assertRaises(ValueError):
            divide(10, 0)

In [38]:
print(TestCalc().test_add())
print(TestCalc().test_multiply())
print(TestCalc().test_divide())

None
None
None


**More Difficult Tests**

In [93]:
class Employee:
    
    raise_amt = 1.05
    
    def __init__(self, first, last, pay):
        self.first = first
        self.last = last
        self.pay = pay
        
    @property
    def email(self):
        return '{}.{}@email.com'.format(self.first, self.last)
    
    @property
    def fullname(self):
        return '{} {}'.format(self.first, self.last)
    
    def apply_raise(self):
        self.pay = int(self.pay * self.raise_amt)
    
    # Mocking (e.g. When a website is down our code will fail, 
    # but code should not fail for this reason)
    def monthly_schedule(self, month):
        response = requests.get('http://company.com/{self.last}/{month}')
        if response.ok:
            return response.text
        else:
            return 'Bad Response'

In [94]:
class TestEmployee(unittest.TestCase):
    
    def test_email(self):
        emp_1 = Employee('John', 'Doe', 50000)
        emp_2 = Employee('Jane', 'Roe', 60000)
        
        self.assertEqual(emp_1.email, 'John.Doe@email.com')
        self.assertEqual(emp_2.email, 'Jane.Roe@email.com')
        
        # Changing first name
        emp_1.first = 'Jane'
        emp_2.first = 'John'
        
        self.assertEqual(emp_1.email, 'Jane.Doe@email.com')
        self.assertEqual(emp_2.email, 'John.Roe@email.com')
        
    def test_fullname(self):
        emp_1 = Employee('John', 'Doe', 50000)
        emp_2 = Employee('Jane', 'Roe', 60000)
        
        self.assertEqual(emp_1.fullname, 'John Doe')
        self.assertEqual(emp_2.fullname, 'Jane Roe')
        
        # Changing first name
        emp_1.first = 'Jane'
        emp_2.first = 'John'
        
        self.assertEqual(emp_1.fullname, 'Jane Doe')
        self.assertEqual(emp_2.fullname, 'John Roe')
        
    def test_apply_raise(self):
        emp_1 = Employee('John', 'Doe', 50000)
        emp_2 = Employee('Jane', 'Roe', 60000)
        
        emp_1.apply_raise()
        emp_2.apply_raise()
        
        self.assertEqual(emp_1.pay, 52500)
        self.assertEqual(emp_2.pay, 63000)

In [95]:
print(TestEmployee().test_email())
print(TestEmployee().test_fullname())
print(TestEmployee().test_apply_raise())

None
None
None


Making code `DRY`

If anything in `emp_1 = Employee('John', 'Doe', 50000)` or `emp_2 = Employee('Jane', 'Roe', 60000)` changes then all the tests needs to be changed.



In [96]:
from unittest.mock import patch 

class TestEmployee(unittest.TestCase):
    
    @classmethod
    def setUpClass(cls):
        print('setUpClass')
        
    @classmethod
    def tearDownClass(cls):
        print('tearDownClass')
    
    def setUp(self):
        self.emp_1 = Employee('John', 'Doe', 50000)
        self.emp_2 = Employee('Jane', 'Roe', 60000)
    
    def tearDown(self):
        pass
    
    def test_email(self):
        self.setUp()
        self.assertEqual(self.emp_1.email, 'John.Doe@email.com')
        self.assertEqual(self.emp_2.email, 'Jane.Roe@email.com')
        
        # Changing first name
        self.emp_1.first = 'Jane'
        self.emp_2.first = 'John'
        
        self.assertEqual(self.emp_1.email, 'Jane.Doe@email.com')
        self.assertEqual(self.emp_2.email, 'John.Roe@email.com')
        
    def test_fullname(self):
        self.setUp()
        self.assertEqual(self.emp_1.fullname, 'John Doe')
        self.assertEqual(self.emp_2.fullname, 'Jane Roe')
        
        # Changing first name
        self.emp_1.first = 'Jane'
        self.emp_2.first = 'John'
        
        self.assertEqual(self.emp_1.fullname, 'Jane Doe')
        self.assertEqual(self.emp_2.fullname, 'John Roe')
        
    def test_apply_raise(self):
        self.setUp()
        self.emp_1.apply_raise()
        self.emp_2.apply_raise()
        
        self.assertEqual(self.emp_1.pay, 52500)
        self.assertEqual(self.emp_2.pay, 63000)
    
    # Mocking not used often
    def test_monthly_schedule(self):
        with patch('requests.get') as mocked_get:
            mocked_get.return_value.ok = True
            mocked_get.return_value.text = 'Success'
            
            schedule = self.emp_1.monthly_schedule('May')
            mocked_get.assert_called_with('http://company/Doe/May')
            self.assertEqual(schedule, 'Success')
            
            mocked_get.return_value.ok = False
            
            schedule = self.emp_2.monthly_schedule('June')
            mocked_get.assert_called_with('http://company/Roe/June')
            self.assertEqual(schedule, 'Bad Response!')

In [97]:
print(TestEmployee().test_email())
print(TestEmployee().test_fullname())
print(TestEmployee().test_apply_raise())
print(TestEmployee().test_monthly_schedule())

None
None
None


ImportError: No module named 'requests'